In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()

In [2]:
from random import randint
filetext = open('text.csv','w')
makanan = ['sate','bakso','mie ayam','gado gado','ayam geprek','tahu tek']
tujuan = ['lapangan sepak bola','mall','pantai','gunung','hutan','isekai']
for x in range(10000):
    if randint(0,1)==0:
        filetext.write('"saya suka makan '+
                       str(makanan[randint(0,len(makanan)-1)])+
                      '",0\n')
    else:
        filetext.write('"saya sering pergi ke '+
                       str(tujuan[randint(0,len(tujuan)-1)])+
                      '",1\n')
filetext.close()

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("text", StringType()),
    StructField("label", IntegerType())
])
txt = spark.read.csv('text.csv', header=False, schema=schema)

In [4]:
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer
wrangled = txt.withColumn('text', regexp_replace(txt.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)
wrangled.show(4, truncate=False)

+---------------------------+-----+---------------------------------+
|text                       |label|words                            |
+---------------------------+-----+---------------------------------+
|saya sering pergi ke isekai|1    |[saya, sering, pergi, ke, isekai]|
|saya suka makan bakso      |0    |[saya, suka, makan, bakso]       |
|saya sering pergi ke isekai|1    |[saya, sering, pergi, ke, isekai]|
|saya suka makan mie ayam   |0    |[saya, suka, makan, mie, ayam]   |
+---------------------------+-----+---------------------------------+
only showing top 4 rows



In [5]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF
txt = wrangled.select('words', 'label')
wrangled = StopWordsRemover(inputCol='words', outputCol='terms').transform(txt)
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024).transform(wrangled)
tf_idf = IDF(inputCol='hash', outputCol='features').fit(wrangled).transform(wrangled)
tf_idf.select('terms', 'features').show(4, truncate=False)

+---------------------------------+--------------------------------------------------------------------------------------------------------------+
|terms                            |features                                                                                                      |
+---------------------------------+--------------------------------------------------------------------------------------------------------------+
|[saya, sering, pergi, ke, isekai]|(1024,[145,298,411,487,965],[0.6862715618238534,2.4687396723244617,0.6862715618238534,0.0,0.6862715618238534])|
|[saya, suka, makan, bakso]       |(1024,[47,398,487,498],[0.6998690518691655,0.6998690518691655,0.0,2.498699971920336])                         |
|[saya, sering, pergi, ke, isekai]|(1024,[145,298,411,487,965],[0.6862715618238534,2.4687396723244617,0.6862715618238534,0.0,0.6862715618238534])|
|[saya, suka, makan, mie, ayam]   |(1024,[47,334,398,487,688],[0.6998690518691655,1.7898614615657151,0.699869051869165

In [6]:
from pyspark.ml.classification import LogisticRegression
txt = tf_idf.select('label', 'features')
txt_train, txt_test = txt.randomSplit([0.8, 0.2], seed=13)
logistic = LogisticRegression(regParam=0.2).fit(txt_train)
prediction = logistic.transform(txt_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|  965|
|    1|       1.0|  974|
+-----+----------+-----+

